@author: Eleni Ntemou
Analysis per day and week on the 5 KPIs 
IPTO: DEV, ANDEV, RMSDEV, NADEV, NRMSDEV
METEOGEN: BIAS, MAE, RMSE, NMAE, NRMSE

DEV = ABS(SUM(MQ-FORECAST)) per every 15 minutes by day sum
ANDEV = DEV/SUM(MQ)
RMSDEV = SUM((ANDEV)^2)
NADEV = ANDEV/SUM(MQ)
NRMSDEV = RMSDEV/SQRT(SUM((MQ)^2))

Libraries

In [289]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
import numpy as np
from datetime import datetime
from matplotlib.dates import WeekdayLocator, DateFormatter
import matplotlib.dates as mdates
from openpyxl import load_workbook

Load the data

E6, MS, MQ from GEARS platform
Manually change path each time when running as the download name is tied to the date that the data was downloaded

In [290]:
path = "C:/Users/Eleni/OneDrive - Hellenic Association for Energy Economics (1)/GEARS TASKS/Tasks/METEOGEN Analysis/met_e6_data.xlsx"
df = pd.read_excel(path)

In [291]:
df = df.set_index('Valid Date UTC+2')

In [292]:
df.head()

,MET Energy (MWh),MET Power (MWh),E6 Power (MW),E6 Energy (MWh),W+1 (MWh),MQ (MWh),MS (MWh),E6-MQ (MWh),MET-MQ (MWh)
Valid Date UTC+2,,,,,,,,,
2023-11-20 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-11-20 02:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-11-20 02:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-11-20 02:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-11-20 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [293]:
# Drop rows where the index is greater than December 18th, 2023
#cutoff_date = pd.to_datetime('2023-12-10 23:45:00')
#df = df[df.index > cutoff_date]
#result = result.drop(result.index[-1])
#result.head()

In [294]:
#Check for nan data
# Check for NaN values
nan_check = df.isna()

any_nan_values = nan_check.any().any()
print(f"Any NaN values: {any_nan_values}")

Any NaN values: False


In [295]:
#fill the missing data with 0
#df = df.fillna(0)

In [296]:
df.columns

Index(['MET Energy (MWh)', 'MET Power (MWh)', 'E6 Power (MW)',
       'E6 Energy (MWh)', 'W+1 (MWh)', 'MQ (MWh)', 'MS (MWh)', 'E6-MQ (MWh)',
       'MET-MQ (MWh)'],
      dtype='object')

In [298]:
df = df.rename(columns={'E6 Energy (MWh)':'Energy E6 (MWh)'})

In [303]:
df = df.rename(columns={'MS (MWh)':'MS (ΜWh)'})

Create the 5 KPIs|
The end file will have different TABS, 1 for each week with the 5 KPIs and 1 TAB with the full range of data 

DAILY

In [299]:
"""
Split by day by day

DEV = ABS(SUM(MQ-FORECAST)) per every 15 minutes by day sum                             MWh
ADEV = SUM(ABS(MQ - FORECAST))                                                          MWh
NADEV = ADEV/(SUM(MQ))                                                                  percentage %
RMSDEV = SQRT(SUM((MQ - FORECAST)^2))                                                   MWh
ANDEV = SUM(DEV)/SUM(MQ) = SUM(ABS(SUM(MQ-FORECAST)))                                   percentage %
NRMSDEV = RMSDEV/SQRT(SUM(MQ^2)) = SQRT(SUM((MQ - FORECAST)^2))/SQRT(SUM(MQ^2))         percentage %           

"""

'\nSplit by day by day\n\nDEV = ABS(SUM(MQ-FORECAST)) per every 15 minutes by day sum                             MWh\nADEV = SUM(ABS(MQ - FORECAST))                                                          MWh\nNADEV = ADEV/(SUM(MQ))                                                                  percentage %\nRMSDEV = SQRT(SUM((MQ - FORECAST)^2))                                                   MWh\nANDEV = SUM(DEV)/SUM(MQ) = SUM(ABS(SUM(MQ-FORECAST)))                                   percentage %\nNRMSDEV = RMSDEV/SQRT(SUM(MQ^2)) = SQRT(SUM((MQ - FORECAST)^2))/SQRT(SUM(MQ^2))         percentage %           \n\n'

DEV

In [300]:
df['MQ - E6'] = df['MQ (MWh)'] - df['Energy E6 (MWh)']
daily_sum = df['MQ - E6'].resample('D').transform('sum')
df['dummy'] = daily_sum
df['E6 DEV'] = np.where(df['dummy'] < 0, -df['dummy'], df['dummy'])

In [301]:
df['MQ - MET'] = df['MQ (MWh)'] - df['MET Energy (MWh)']
daily_sum = df['MQ - MET'].resample('D').transform('sum')
df['dummy'] = daily_sum
df['MET DEV'] = np.where(df['dummy'] < 0, -df['dummy'], df['dummy'])

In [304]:
df['MQ - MS'] = df['MQ (MWh)'] - df['MS (ΜWh)']
daily_sum = df['MQ - MS'].resample('D').transform('sum')
df['dummy'] = daily_sum
df['MS DEV'] = np.where(df['dummy'] < 0, -df['dummy'], df['dummy'])

In [305]:
#df['MQ - W+1'] = df['MQ (MWh)'] - df[' IPTO W+1 (MWh)']
#daily_sum = df['MQ - W+1'].resample('D').transform('sum')
#df['dummy'] = daily_sum
#df['W+1 DEV'] = np.where(df['dummy'] < 0, -df['dummy'], df['dummy'])

ADEV

In [306]:
df['MQ - E6'] = np.where(df['MQ - E6'] < 0, -df['MQ - E6'], df['MQ - E6'])
daily_sum = df['MQ - E6'].resample('D').transform('sum')
df['E6 ADEV'] = daily_sum

In [307]:
df['MQ - MET'] = np.where(df['MQ - MET'] < 0, -df['MQ - MET'], df['MQ - MET'])
daily_sum = df['MQ - MET'].resample('D').transform('sum')
df['MET ADEV'] = daily_sum

In [308]:
df['MQ - MS'] = np.where(df['MQ - MS'] < 0, -df['MQ - MS'], df['MQ - MS'])
daily_sum = df['MQ - MS'].resample('D').transform('sum')
df['MS ADEV'] = daily_sum

In [309]:
#df['MQ - W+1'] = np.where(df['MQ - W+1'] < 0, -df['MQ - W+1'], df['MQ - W+1'])
#daily_sum = df['MQ - W+1'].resample('D').transform('sum')
#df['W+1 ADEV'] = daily_sum

NADEV

In [310]:
daily_sum = df['MQ (MWh)'].resample('D').transform('sum')
df['DAILY SUM MQ'] = daily_sum
df['E6 NADEV'] = (df['E6 ADEV']/df['DAILY SUM MQ'])*100

In [311]:
df['MET NADEV'] = (df['MET ADEV']/df['DAILY SUM MQ'])*100

In [312]:
df['MS NADEV'] = (df['MS ADEV']/df['DAILY SUM MQ'])*100

In [313]:
#df['W+1 NADEV'] = (df['W+1 ADEV']/df['DAILY SUM MQ'])*100

RMSDEV

In [314]:
daily_sum = ((df['MQ - E6'])**2).resample('D').transform('sum')
df['E6 RMSDEV'] = (daily_sum)**(1/2)

In [315]:
daily_sum = ((df['MQ - MET'])**2).resample('D').transform('sum')
df['MET RMSDEV'] = (daily_sum)**(1/2)

In [316]:
daily_sum = ((df['MQ - MS'])**2).resample('D').transform('sum')
df['MS RMSDEV'] = (daily_sum)**(1/2)

In [317]:
#daily_sum = ((df['MQ - W+1'])**2).resample('D').transform('sum')
#df['W+1 RMSDEV'] = (daily_sum)**(1/2)

ANDEV

In [318]:
df['E6 ANDEV'] = (df['E6 DEV']/df['DAILY SUM MQ'])*100

In [319]:
df['MET ANDEV'] = (df['MET DEV']/df['DAILY SUM MQ'])*100

In [320]:
df['MS ANDEV'] = (df['MS DEV']/df['DAILY SUM MQ'])*100

In [321]:
#df['W+1 ANDEV'] = (df['W+1 DEV']/df['DAILY SUM MQ'])*100

NRMSDEV

In [322]:
daily_sum = ((df['MQ (MWh)'])**2).resample('D').transform('sum')
df['dummy'] = (daily_sum)**(1/2)
df['E6 NRMSDEV'] = (df['E6 RMSDEV']/df['dummy'])*100

In [323]:
df['MET NRMSDEV'] = (df['MET RMSDEV']/df['dummy'])*100

In [324]:
df['MS NRMSDEV'] = (df['MS RMSDEV']/df['dummy'])*100

In [325]:
#df['W+1 NRMSDEV'] = (df['W+1 RMSDEV']/df['dummy'])*100

Rename some columns

In [326]:
mapping = {'E6 DEV': 'E6 DEV (MWh)',
'MET DEV': 'MET DEV (MWh)',
#'W+1 DEV': 'W+1 DEV (MWh)',
'MS DEV': 'MS DEV (MWh)',
'E6 ADEV': 'E6 ADEV (MWh)',
'MET ADEV': 'MET ADEV (MWh)',
'MS ADEV': 'MS ADEV (MWh)',
#'W+1 ADEV': 'W+1 ADEV (MWh)',
'E6 NADEV': 'E6 NADEV (%)',
'MET NADEV': 'MET NADEV (%)',
'MS NADEV': 'MS NADEV (%)',
#'W+1 NADEV': 'W+1 NADEV (%)',
'E6 RMSDEV': 'E6 RMSDEV (MWh)',
'MET RMSDEV': 'MET RMSDEV (MWh)',
'MS RMSDEV': 'MS RMSDEV (MWh)',
#'W+1 RMSDEV': 'W+1 RMSDEV (MWh)',
'E6 NRMSDEV': 'E6 NRMSDEV (%)',
'MS NRMSDEV': 'MS NRMSDEV (%)',
#'W+1 NRMSDEV':  'W+1 NRMSDEV(%)',
'E6 ANDEV': "E6 ANDEV (%)",
'MS ANDEV': "MS ANDEV (%)",
'MET ANDEV': 'MET ANDEV (%)',
#'W+1 ANDEV': 'W+1 ANDEV (%)',
'MET NRMSDEV': 'MET NRMSDEV (%)'}

In [327]:
df.rename(columns = mapping, inplace=True)

Save to new dataframe the 5 KPIs

In [328]:
mask = df.index.time == pd.to_datetime('00:00:00').time()

In [329]:
#df.columns

In [330]:
#columns_to_select = ['E6 DEV (MWh)', 'MET DEV (MWh)', 'MS DEV (MWh)',
#       'W+1 DEV (MWh)', 'E6 ADEV (MWh)', 'MET ADEV (MWh)', 'MS ADEV (MWh)',
#       'W+1 ADEV (MWh)', 'DAILY SUM MQ', 'E6 NADEV (%)', 'MET NADEV (%)',
#       'MS NADEV (%)', 'W+1 NADEV (%)', 'E6 RMSDEV (MWh)', 'MET RMSDEV (MWh)',
#       'MS RMSDEV (MWh)', 'W+1 RMSDEV (MWh)', 'E6 ANDEV (%)', 'MET ANDEV (%)',
#       'MS ANDEV (%)', 'W+1 ANDEV (%)', 'E6 NRMSDEV (%)', 'MET NRMSDEV (%)',
#       'MS NRMSDEV (%)', 'W+1 NRMSDEV(%)']
columns_to_select = ['E6 DEV (MWh)', 'MET DEV (MWh)', 'MS DEV (MWh)',
       'E6 ADEV (MWh)', 'MET ADEV (MWh)', 'MS ADEV (MWh)',
       'DAILY SUM MQ', 'E6 NADEV (%)', 'MET NADEV (%)',
       'MS NADEV (%)', 'E6 RMSDEV (MWh)', 'MET RMSDEV (MWh)',
       'MS RMSDEV (MWh)', 'E6 ANDEV (%)', 'MET ANDEV (%)',
       'MS ANDEV (%)', 'E6 NRMSDEV (%)', 'MET NRMSDEV (%)',
       'MS NRMSDEV (%)']

In [331]:
new_df = df[mask][columns_to_select]

Save the data to an excel file with different sheets

In [334]:
path = "C:/Users/Eleni/OneDrive - Hellenic Association for Energy Economics (1)/GEARS TASKS/Tasks/METEOGEN Analysis/weekly_KPIs.xlsx"

In [335]:
# Sort the DataFrame by the datetime index (if it's not already sorted)
new_df = new_df.sort_index()

# Group the DataFrame by week
weekly_groups = new_df.groupby(pd.Grouper(freq='W'))

# Create an Excel writer with the xlsxwriter engine
excel_writer = pd.ExcelWriter('path', engine='xlsxwriter')

# Iterate through the groups and save each week as a separate sheet
for week, week_data in weekly_groups:
    sheet_name = f'Week_{week.strftime("%Y-%m-%d")}'
    week_data.to_excel(excel_writer, sheet_name=sheet_name)

# Save the Excel file
excel_writer.close()

WEEKLY

In [351]:
df_new = pd.DataFrame()

MQ

In [352]:
weekly_sum = (df['MQ (MWh)']).resample('W').transform('sum')
df_new['weekly MQ'] = weekly_sum

ADEV

In [353]:
#df['MQ - E6'] = np.where(df['MQ - E6'] < 0, -df['MQ - E6'], df['MQ - E6'])
weekly_sum = df['MQ - E6'].resample('W').transform('sum')
df_new['weekly E6 ADEV'] = weekly_sum

In [354]:
weekly_sum = df['MQ - MET'].resample('W').transform('sum')
df_new['weekly MET ADEV'] = weekly_sum

In [355]:
weekly_sum = df['MQ - MS'].resample('W').transform('sum')
df_new['weekly MS ADEV'] = weekly_sum

NADEV

In [356]:
df_new['weekly E6 NADEV'] = (df_new['weekly E6 ADEV']/df_new['weekly MQ'])*100
df_new['weekly MET NADEV'] = (df_new['weekly MET ADEV']/df_new['weekly MQ'])*100
df_new['weekly MS NADEV'] = (df_new['weekly MS ADEV']/df_new['weekly MQ'])*100

RMSDEV

In [357]:
weekly_sum = ((df['MQ - E6'])**2).resample('W').transform('sum')
df_new['weekly E6 RMSDEV'] = (weekly_sum)**(1/2)
weekly_sum = ((df['MQ - MET'])**2).resample('W').transform('sum')
df_new['weekly MET RMSDEV'] = (weekly_sum)**(1/2)
weekly_sum = ((df['MQ - MS'])**2).resample('W').transform('sum')
df_new['weekly MS RMSDEV'] = (weekly_sum)**(1/2)

NRMSDEV

In [358]:
weekly_sum = ((df['MQ (MWh)'])**2).resample('W').transform('sum')
df['dummy'] = (weekly_sum)**(1/2)
df_new['weekly E6 NRMSDEV'] = (df_new['weekly E6 RMSDEV']/df['dummy'])*100
df_new['weekly MET NRMSDEV'] = (df_new['weekly MET RMSDEV']/df['dummy'])*100
df_new['weekly MS NRMSDEV'] = (df_new['weekly MS RMSDEV']/df['dummy'])*100

Filter and keep only Monday and the first entry

In [359]:
df_new = df_new[(df_new.index.time == pd.to_datetime('02:00:00').time()) & (df_new.index.day_name() == 'Monday')]

In [360]:
df_new.head()

,weekly MQ,weekly E6 ADEV,weekly MET ADEV,weekly MS ADEV,weekly E6 NADEV,weekly MET NADEV,weekly MS NADEV,weekly E6 RMSDEV,weekly MET RMSDEV,weekly MS RMSDEV,weekly E6 NRMSDEV,weekly MET NRMSDEV,weekly MS NRMSDEV
Valid Date UTC+2,,,,,,,,,,,,,
2023-11-20 02:00:00,202.146,67.14275,83.018299,67.964,33.214978,41.068484,33.621244,5.394383,5.949430,5.788067,31.586558,34.836607,33.891755
2023-11-27 02:00:00,230.384,83.12275,48.809709,66.732,36.080088,21.186241,28.965553,5.878018,3.848615,5.552349,31.158788,20.401125,29.432450
2023-12-04 02:00:00,189.164,76.37550,69.830889,72.480,40.375283,36.915528,38.315959,6.633385,6.010473,6.315935,41.356477,37.472878,39.377310
2023-12-11 02:00:00,207.018,81.63350,63.129325,57.438,39.433044,30.494607,27.745413,6.195611,5.354354,4.789610,34.953948,30.207802,27.021670


SAVE TO DIFFERENT FILE

In [361]:
path = "C:/Users/Eleni/OneDrive - Hellenic Association for Energy Economics (1)/GEARS TASKS/Tasks/METEOGEN Analysis/KPIs_for_ther_week.xlsx"

In [362]:
#data = pd.read_excel(path)
#data.info()

In [363]:
#data.set_index('Valid Date UTC+2', inplace = True)

In [364]:
#df_new = pd.concat([df_new, data], axis = 0)

In [365]:
#df_new.head()

In [366]:
df_new.to_excel(path,index=True)

TOLLERANCE

NCBALR_C1p,m=0 only if both A & B ≤ 0, otherwise =MAX(A,B) <br>
NCBALR_C2p,m=0 only if ANDEV < 𝑻𝑶𝑳𝒓,𝑫𝑬𝑽_NORM <br>
NADEV < 𝑻𝑶𝑳𝒓,𝑨𝑫𝑬𝑽 (%) <br>
NRMSDEV < 𝑻𝑶𝑳𝒓, 𝑹𝑴𝑺𝑫𝑬𝑽 (%) <br>

Α = (𝑈𝑁𝐶𝐵𝐴𝐿𝑅𝐴𝐷𝐸𝑉 ∗ 𝐴𝐷𝐸𝑉𝑝,𝑚) ∗ (𝑁𝐴𝐷𝐸𝑉𝑝,𝑚 − 𝑇𝑂𝐿𝑟,𝐴𝐷𝐸𝑉) (€) <br>
Β = (𝑈𝑁𝐶𝐵𝐴𝐿𝑅𝑅𝑀𝑆𝐷𝐸𝑉 ∗ 𝑅𝑀𝑆𝐷𝐸𝑉𝑝,𝑚) ∗ (𝑁𝑅𝑀𝑆𝐷𝐸𝑉𝑝,𝑚 − 𝑇𝑂𝐿𝑟,𝑅𝑀𝑆𝐷𝐸𝑉) (€)


These constants can change, they depend from IPTO <br>
α2ADEV = -0.009 <br>
α2RMSDEV = -0.009 <br>
α3ADEV = 0.28 <br>
α3RMSDEV = 0.28 <br>
α1ADEV = 0.35 <br>
α1RMSDEV = 0.4 <br>
UNCBALR, ADEV  €/MWh = 10 <br>
UNCBALR, RMSDEV €/MWh = 210 <br>
minTOLADEV =20% <br>
minTOLRMSDEV = 20% <br>
maxTOLADEV = 100% <br>
maxTOLRMSDEV = 100% <br>
maxTOL_DEV_NORM = 0.27<br>
minTOL_DEV_NORM = 0.0109<br>
a1DEV_NORM = 0.27<br>
a2DEV_NORM = 0.0109<br>
a3DEV_NORM = 0.28<br>

In [367]:
df_tol = pd.DataFrame()

In [368]:
#Set the constants
a1ADEV = 0.35
a1RMSDEV = 0.4
a2ADEV = -0.009
a2RMSDEV = a2ADEV
a3ADEV = 0.28
a3RMSDEV = a3ADEV
UNCBALR_ADEV = 10 #EURO/MWh
UNCBALR_RMSDEV = 210 #EURO/MWh
min_TOLADEV = 0.20 #20%
minTOLRMSDEV = 0.20#20%
maxTOLADEV = 1#100%
maxTOLRMSDEV = 1#100%

In [369]:
maxTOL_DEV_NORM = 1#100%
minTOL_DEV_NORM = 0.0005#0.05%
a1DEV_NORM = 0.27
a2DEV_NORM = -0.0109
a3DEV_NORM = 0.28

TOL_ADEV<BR>
𝑻𝑶𝑳𝒓,𝑨𝑫𝑬𝑽 (%) = MAX(minTOLADEV,MIN(maxTOLADEV, α1ADEV+ α2ADEV*MQ^ α3ADEV))->

In [370]:
def calculate_TOL_ADEV(x):
    return max(min_TOLADEV, min(maxTOLADEV, (a1ADEV + (a2ADEV * (x ** a3ADEV)))))

# Apply the function to each element of the 'MQ weekly' column
df_tol['TOL_ADEV'] = df_new['weekly MQ'].apply(calculate_TOL_ADEV)

TOL_DEV_NORM<BR>
𝑻𝑶𝑳𝒓,𝑫𝑬𝑽_NORM (%) = MAX(minTOL DEV_NORM, MIN(maxTOL DEV_NORM, α1DEV_NORM+ α2DEV_NORM *MQ^ α3DEV_NORM)<br>
ΕΙΝΑΙ ΕΝΤΑΞΕΙ

In [371]:
def calculate_TOL_DEV_NORM(x):
    return max(minTOL_DEV_NORM, min(maxTOL_DEV_NORM, a1DEV_NORM + a2DEV_NORM * (x ** a3DEV_NORM)))

# Apply the function to each element of the 'MQ weekly' column
df_tol['TOL_DEV_NORM'] = df_new['weekly MQ'].apply(calculate_TOL_DEV_NORM)

TOL_RMSDEV<BR>
𝑻𝑶𝑳𝒓,𝑹𝑴𝑺𝑫𝑬𝑽 (%) = MAX(minTOLRMSDEV,MIN(maxTOLRMSDEV, α1RMSDEV+ α2RMSDEV*MQ^ α3RMSDEV))

In [372]:
def calculate_TOL_RMSDEV(x):
    return max(minTOLRMSDEV, min(maxTOLRMSDEV, a1RMSDEV + a2RMSDEV * (x ** a3RMSDEV)))

# Apply the function to each element of the 'MQ weekly' column
df_tol['TOL_RMSDEV'] = df_new['weekly MQ'].apply(calculate_TOL_RMSDEV)

Α = (UNCBALR, ADEV  €/MWh * ADEV) * (NADEV - 𝑻𝑶𝑳𝒓,𝑨𝑫𝑬𝑽 (%)) (€)

In [373]:
df_tol['E6 A'] = (UNCBALR_ADEV*df_new['weekly E6 ADEV'])*(df_new['weekly E6 NADEV']/100-df_tol['TOL_ADEV'])

In [374]:
df_tol['MET A'] = (UNCBALR_ADEV* df_new['weekly MET ADEV'])*(df_new['weekly MET NADEV']/100-df_tol['TOL_ADEV'])

In [375]:
df_tol['MS A'] = UNCBALR_ADEV* df_new['weekly MS ADEV']*(df_new['weekly MS NADEV']/100-df_tol['TOL_ADEV'])

B = (UNCBALR, RMSDEV €/MWh * RMSDEV) * (NRMSDEV - 𝑻𝑶𝑳𝒓,𝑹𝑴𝑺𝑫𝑬𝑽 (%))(€)

In [376]:
df_tol['E6 B'] = UNCBALR_RMSDEV* df_new['weekly E6 RMSDEV']*(df_new['weekly E6 NRMSDEV']/100-df_tol['TOL_RMSDEV'])

In [377]:
df_tol['MET B'] = UNCBALR_RMSDEV* df_new['weekly MET RMSDEV']*(df_new['weekly MET NRMSDEV']/100-df_tol['TOL_RMSDEV'])

In [378]:
df_tol['MS B'] = UNCBALR_RMSDEV* df_new['weekly MS RMSDEV']*(df_new['weekly MS NRMSDEV']/100-df_tol['TOL_RMSDEV'])

In [379]:
df_tol.head(10)

,TOL_ADEV,TOL_DEV_NORM,TOL_RMSDEV,E6 A,MET A,MS A,E6 B,MET B,MS B
Valid Date UTC+2,,,,,,,,,
2023-11-20 02:00:00,0.310205,0.221804,0.360205,14.734346,83.416676,17.675712,-50.228580,-14.791299,-25.874743
2023-11-27 02:00:00,0.308721,0.220007,0.358721,43.290228,-47.276397,-12.722769,-58.180359,-125.037840,-75.085894
2023-12-04 02:00:00,0.310938,0.222691,0.360938,70.887982,40.653817,52.346389,73.310005,17.407032,43.551048
2023-12-11 02:00:00,0.309939,0.221481,0.359939,68.891915,-3.151831,-18.658498,-13.530234,-65.059405,-90.244026


In [380]:
# Dropping the last row
#df_tol = df_tol.drop(df_tol.index[-1])

In [381]:
# Dropping the last row
#df_new = df_new.drop(df_new.index[-1])

In [382]:
nan_values = df_new.isna()
# Printing the positions of True values
true_positions = nan_values.stack().loc[lambda x: x].index
print("\nPositions of True values:")
print(true_positions)


Positions of True values:
MultiIndex([], names=['Valid Date UTC+2', None])


NCBALR_C1 = 0 <-> A & B <=0<BR>
ELSE|    NCBALR_C1 = MAX(A,B)

In [383]:
if (df_tol['E6 A'] <= 0).all() and (df_tol['E6 B'] <= 0).all():
    df_tol['NCBALR_C1 E6'] = 0
else:
     df_tol['NCBALR_C1 E6'] = df_tol[['E6 A', 'E6 B']].max(axis=1)

In [384]:
if (df_tol['MET A'] <= 0).all() and (df_tol['MET B'] <= 0).all():
    df_tol['NCBALR_C1 MET'] = 0
else:
     df_tol['NCBALR_C1 MET'] = df_tol[['MET A', 'MET B']].max(axis=1)


In [385]:
if (df_tol['MS A'] <= 0).all() and (df_tol['MS B'] <= 0).all():
    df_tol['NCBALR_C1 MS'] = 0
else:
     df_tol['NCBALR_C1 MS'] = df_tol[['MS A', 'MS B']].max(axis=1)


NCBALR_C2 = 0 <-> ANDEV < 𝑻𝑶𝑳𝒓,𝑫𝑬𝑽_NORM (%)

In [386]:
if (df_new['weekly E6 ADEV'] < df_tol['TOL_DEV_NORM']).all():
    df_tol['NCBALR_C2 E6'] = 0
else:
    df_tol['NCBALR_C2 E6'] = -1

In [387]:
if (df_new['weekly MET ADEV'] < df_tol['TOL_DEV_NORM']).all():
    df_tol['NCBALR_C2 MET'] = 0
else:
    df_tol['NCBALR_C2 MET'] = -1

In [388]:
if (df_new['weekly MS ADEV'] < df_tol['TOL_DEV_NORM']).all():
    df_tol['NCBALR_C2 MS'] = 0
else:
    df_tol['NCBALR_C2 MS'] = -1

In [389]:
df_tol = df_tol.sort_index()
df_tol.tail()

,TOL_ADEV,TOL_DEV_NORM,TOL_RMSDEV,E6 A,MET A,MS A,E6 B,MET B,MS B,NCBALR_C1 E6,NCBALR_C1 MET,NCBALR_C1 MS,NCBALR_C2 E6,NCBALR_C2 MET,NCBALR_C2 MS
Valid Date UTC+2,,,,,,,,,,,,,,,
2023-11-20 02:00:00,0.310205,0.221804,0.360205,14.734346,83.416676,17.675712,-50.228580,-14.791299,-25.874743,14.734346,83.416676,17.675712,-1,-1,-1
2023-11-27 02:00:00,0.308721,0.220007,0.358721,43.290228,-47.276397,-12.722769,-58.180359,-125.037840,-75.085894,43.290228,-47.276397,-12.722769,-1,-1,-1
2023-12-04 02:00:00,0.310938,0.222691,0.360938,70.887982,40.653817,52.346389,73.310005,17.407032,43.551048,73.310005,40.653817,52.346389,-1,-1,-1
2023-12-11 02:00:00,0.309939,0.221481,0.359939,68.891915,-3.151831,-18.658498,-13.530234,-65.059405,-90.244026,68.891915,-3.151831,-18.658498,-1,-1,-1


In [390]:
path = "C:/Users/Eleni/OneDrive - Hellenic Association for Energy Economics (1)/GEARS TASKS/Tasks/METEOGEN Analysis/tolerance.xlsx"

In [391]:
# Create an ExcelWriter object
writer = pd.ExcelWriter(path, engine='xlsxwriter')

In [392]:
# Iterate over each week (Monday to Sunday) and write to a separate sheet
for start_date, groups in df_tol.groupby(pd.Grouper(freq='W-Mon', closed='left', label='left')):
    end_date = start_date + pd.DateOffset(days=6)
    
    sheet_name = f'{start_date.strftime("%Y-%m-%d")}_{end_date.strftime("%Y-%m-%d")}'
    
    groups.to_excel(writer, header= True, sheet_name=sheet_name)

# Close the ExcelWriter, and the file will be saved automatically
writer.close()